# "Bookmakers do not want to limit the amount of money that is being bet. They want to balance the earnings with the givings."

###                                                         We want to undertand how bookmakers
#### 1) calculate the odds between two teams (suppose there is no tie)
#### 2) change the odds accordingly to the amount that is being played among those two choices
#### 3) change the odds based on real-time occurancies (e.g. injuries/red cards, new coach/player, form etc.)


### Below i've written a class that
#### - Calculates the odd of a team based on (total money/ bets on that team) * 1 - the margin that bookmakers set as profit
#### - Accepts the bets
#### - Calculates the total money won
#### - Recalculate the odds after new bets

In [149]:
import random


class DynamicBookmaker:
    def __init__(self, initial_odds=(2.0, 2.0), margin=0.05):
        self.margin = margin
        self.bets = {'TeamA': 0, 'TeamB': 0}
        self.odds = list(initial_odds)

    def calculate_odds(self):
        total_money = sum(self.bets.values())
        if self.bets['TeamB'] == 0 or self.bets['TeamA']==0 :
            return 0
            
        else:
            odds_team_a = (total_money / (self.bets['TeamA'] )) * (1 - self.margin)
            odds_team_b = (total_money / (self.bets['TeamB'] )) * (1 - self.margin)
            return odds_team_a, odds_team_b
    def take_bet(self, team, amount):
        if team in self.bets:
            self.bets[team] += amount
            self.update_odds()

    def settle_bets(self, winner):
        if winner in self.bets:
            if winner == 'TeamA':
                team_w = 0
            else:
                team_w = 1
            total_money_won = self.bets[winner] * self.odds[team_w]
            
            self.update_odds()
            return total_money_won
        else:
            print("Invalid winner.")
            return 0

    def update_odds(self):
        self.odds = self.calculate_odds()



#### Now, think that some experts give the initial odds, but still the odds are calculated by the amount of  money that is being bet.


In [154]:
import random
import sys

class DynamicBookmakerV2:
    def __init__(self, initial_odds, margin):
        self.margin = margin
        self.bets = {'TeamA': 1, 'TeamB': 1}
        self.odds = list(initial_odds)
        print(f'Initial odds TeamA: {initial_odds[0]}, TeamB = {initial_odds[1]}\nCommission by house: {margin}')

    def calculate_odds(self):
        total_money = sum(self.bets.values())
        if self.bets['TeamB'] == 0 or self.bets['TeamA']==0 :
            return 0
            
        else:
            odds_team_a = (total_money / (self.bets['TeamA'] )) * (1 - self.margin)
            odds_team_b = (total_money / (self.bets['TeamB'] )) * (1 - self.margin)
            if odds_team_a < 1.01:
                odds_team_a = 1.01
            if odds_team_b < 1.01:
                odds_team_b = 1.01
            if odds_team_a > 11.13:
                odds_team_a = 11.13
            if odds_team_b > 11.13:
                odds_team_b = 11.13
                
            return odds_team_a, odds_team_b

    def take_bet(self, team, amount):
        if team in self.bets:
            if amount > 5000:
                
                sys.exit("Max bet is 5000")   
            else:
                self.bets[team] += amount
                self.update_odds()

    def settle_bets(self, winner):
        if winner in self.bets:
            if winner == 'TeamA':
                team_w = 0
            else:
                team_w = 1
            total_money_won = self.bets[winner] * self.odds[team_w]
            
            self.update_odds()
            return total_money_won
        else:
            print("Invalid winner.")
            return 0

    def update_odds(self):
        self.odds = self.calculate_odds()


# Example usage:
bookmaker = DynamicBookmakerV2((2,2),0.07)


Initial odds TeamA: 2, TeamB = 2
Commission by house: 0.07


#### When executing the code below we simulate a betting process. 

In [158]:
# Simulation: Taking bets

team = input('You want to bet on team A or B: ')
money = input('How much money do you want to bet? ')
bookmaker.take_bet(f'Team{team}', int(money))


# Simulation: Retrieving and printing updated odds
odds_team_a, odds_team_b = bookmaker.odds
print(f"Team A Odds: {odds_team_a:.2f}, Team B Odds: {odds_team_b:.2f}")

# Simulation: Settling bets
winner = random.choice(['TeamA', 'TeamB'])
print(f"Winner: {winner}")
total_money_won = bookmaker.settle_bets(winner)

print(f"Total money that were bet: {sum(bookmaker.bets.values())}")
print(f"Total money won by players: {total_money_won}")
print(f"Commission ({bookmaker.margin}%) for bookmakers: {sum(bookmaker.bets.values())-total_money_won}")

You want to bet on team A or B: B
How much money do you want to bet? 5
Team A Odds: 1.86, Team B Odds: 1.86
Winner: TeamA
Total money that were bet: 12
Total money won by players: 11.16
Commission (0.07%) for bookmakers: 0.8399999999999999
